In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
dataset_dir = 'eeg-feature-generation/dataset/original_data'

datasets = os.listdir(dataset_dir)
datasets_df = pd.DataFrame({'filenames': datasets})

In [3]:
datasets_df['filenames']

0         name-concentrating-1.csv
1     subjecta-concentrating-1.csv
2     subjecta-concentrating-2.csv
3           subjecta-neutral-1.csv
4           subjecta-neutral-2.csv
5           subjecta-relaxed-1.csv
6           subjecta-relaxed-2.csv
7     subjectb-concentrating-1.csv
8     subjectb-concentrating-2.csv
9           subjectb-neutral-1.csv
10          subjectb-neutral-2.csv
11          subjectb-relaxed-1.csv
12          subjectb-relaxed-2.csv
13    subjectc-concentrating-1.csv
14    subjectc-concentrating-2.csv
15          subjectc-neutral-1.csv
16          subjectc-neutral-2.csv
17          subjectc-relaxed-1.csv
18          subjectc-relaxed-2.csv
19    subjectd-concentrating-1.csv
20    subjectd-concentrating-2.csv
21          subjectd-neutral-1.csv
22          subjectd-neutral-2.csv
23          subjectd-relaxed-1.csv
24          subjectd-relaxed-2.csv
Name: filenames, dtype: object

In [83]:
classes = ['concentrating', 'neutral', 'relaxed']
datasets_by_class = {'concentrating':[], 'neutral':[], 'relaxed':[]}
window_size = 256 #samples


for class_name in classes:
    datasets = datasets_df[datasets_df['filenames'].str.contains(class_name)]

    for filename in datasets['filenames']:
        sub_df = pd.read_csv(f"{dataset_dir}/{filename}") 

        windows = []
        cur_window = []
        sample_count = 0

        for row in range(sub_df.shape[0]):
            sample_count+=1
            cur_window.append(sub_df.iloc[row])

            if sample_count == window_size:
                windows.append(cur_window)
                cur_window = []
                sample_count = 0

        # datasets_by_class[class_name].append(np.array(windows))
        datasets_by_class[class_name]+=windows

In [84]:
concentrating_data = np.asarray(datasets_by_class['concentrating'], dtype='object')
neutral_data = np.asarray(datasets_by_class['neutral'], dtype='object')
relaxed_data = np.asarray(datasets_by_class['relaxed'], dtype='object')

In [85]:
print(concentrating_data.shape)
print(neutral_data.shape)
print(relaxed_data.shape)

(423, 256, 6)
(422, 256, 6)
(454, 256, 6)


In [107]:
matrix = concentrating_data[0][:,1]
print(matrix.shape)

(256,)


In [111]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define input shape (256x6 matrix with a single channel)
input_shape = (256, 6)

# Build the CNN model
model = keras.Sequential([
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Change activation for multi-class classification if needed
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


ModuleNotFoundError: No module named 'tensorflow'